In [17]:
import pandas as pd

# Carregar os dados
df = pd.read_excel('Dados.xlsx', engine='openpyxl')

# Filtrar para Brasil e Grupo 6
df_brasil_grupo6 = df[(df['mercado'] == 'BRASIL') & (df['Grupo de Produto'] == 'Grupo 6')]

# Exibir as primeiras linhas para verificar
print(df_brasil_grupo6.head())


                        db-id control_id                 invite_id  \
50   620a6b046c698d64297d3189     0JQS72  620a64bf115c606481d093bc   
53   620a6be3a1525664556359ef     1X5TC6  620a608ab0d07664995bec8d   
107  620a8da5ac9ce10d742e8993     7Z0HX5  620a63685e1e5064865de50c   
144  620ae6227324330db5ed0d74     UZYIU6  620a6088b0d07664995b9a77   
232  620eb93c62ab432755237eb0     X90B3C  620a63695e1e5064865e2865   

    Tipo Produto Grupo de Produto Familia do Produto Modelo do Produto  \
50        Tipo 1          Grupo 6         Familia 15         Modelo 60   
53        Tipo 1          Grupo 6         Familia 15         Modelo 60   
107       Tipo 1          Grupo 6         Familia 15         Modelo 60   
144       Tipo 1          Grupo 6         Familia 15         Modelo 60   
232       Tipo 1          Grupo 6         Familia 15         Modelo 52   

    Chassi do Produto Marca do Produto           nome  ...  \
50           Chassi 1          Marca 2   Cliente 1244  ...   
53        

In [22]:
import pandas as pd

# Carregar os dados
df = pd.read_excel('Dados.xlsx', engine='openpyxl')

# Filtrar para Brasil e Grupo 6
df_brasil_grupo6 = df[(df['mercado'] == 'BRASIL') & (df['Grupo de Produto'] == 'Grupo 6')].copy()

# Função para categorizar a nota
def categorize_target(nota):
    if nota >= 9:
        return 'Promotor'
    elif nota >= 7:
        return 'Neutro'
    else:
        return 'Detrator'

# Adicionar a coluna 'target' categorizando pela nota usando .loc
df_brasil_grupo6.loc[:, 'target'] = df_brasil_grupo6['nota'].apply(categorize_target)

# Verificar as primeiras linhas com a nova coluna
print(df_brasil_grupo6[['nota', 'target']].head())


     nota    target
50     10  Promotor
53     10  Promotor
107     8    Neutro
144     2  Detrator
232     8    Neutro


In [24]:
# Mapear siglas dos estados para regiões
regions_siglas = {
    'sul': ['PR', 'SC', 'RS'],
    'sudeste': ['SP', 'RJ', 'MG', 'ES'],
    'centro-oeste': ['GO', 'MT', 'MS', 'DF'],
    'nordeste': ['BA', 'PE', 'CE', 'RN', 'PB', 'SE', 'AL', 'MA', 'PI'],
    'norte': ['AM', 'PA', 'AC', 'RO', 'TO', 'RR', 'AP']
}

def assign_region_from_sigla(estado):
    for region, states in regions_siglas.items():
        if estado in states:
            return region
    return 'desconhecido'

# Certifique-se de estar aplicando no DataFrame correto
df_brasil_grupo6['região'] = df_brasil_grupo6['estado'].apply(assign_region_from_sigla)

# Verificar os resultados
print(df_brasil_grupo6[['estado', 'região']].drop_duplicates())



      estado        região
50        **  desconhecido
107       PI      nordeste
144       SP       sudeste
232       MG       sudeste
261      NaN  desconhecido
315       PA         norte
397       MT  centro-oeste
1014      TO         norte
1256      PR           sul
1628      GO  centro-oeste
2006      MS  centro-oeste
2110      SC           sul
2283      MA      nordeste
2422      BA      nordeste
3261      RS           sul
8614      RO         norte
11552     DF  centro-oeste


In [26]:
# Criar a coluna 'safra' com o ano da data de resposta para o Grupo 6
df_brasil_grupo6['safra'] = pd.to_datetime(df_brasil_grupo6['data_resposta'], errors='coerce', dayfirst=True).dt.year

# Verificar o resultado
print(df_brasil_grupo6[['data_resposta', 'safra']].head())


           data_resposta  safra
50   14/02/2022 11:45:24   2022
53   14/02/2022 11:49:07   2022
107  14/02/2022 14:13:09   2022
144  14/02/2022 20:30:42   2022
232  17/02/2022 18:08:12   2022


In [27]:
# Calcular a volumetria do target
volumetria_target = df_brasil_grupo6.groupby('target').size().reset_index(name='Total')
volumetria_target['Percentual'] = (volumetria_target['Total'] / volumetria_target['Total'].sum()) * 100

# Exibir a volumetria
print("\nVolumetria por Target:")
print(volumetria_target)



Volumetria por Target:
     target  Total  Percentual
0  Detrator     50   10.940919
1    Neutro     91   19.912473
2  Promotor    316   69.146608


In [28]:
# Calcular volumetria por safra
volumetria_safra = df_brasil_grupo6.groupby(['safra', 'target']).size().unstack(fill_value=0)
volumetria_safra['Total'] = volumetria_safra.sum(axis=1)
volumetria_safra['%Promotor'] = (volumetria_safra['Promotor'] / volumetria_safra['Total']) * 100
volumetria_safra['%Neutro'] = (volumetria_safra['Neutro'] / volumetria_safra['Total']) * 100
volumetria_safra['%Detrator'] = (volumetria_safra['Detrator'] / volumetria_safra['Total']) * 100

# Exibir a volumetria
print("\nVolumetria por Safra:")
print(volumetria_safra)



Volumetria por Safra:
target  Detrator  Neutro  Promotor  Total  %Promotor    %Neutro  %Detrator
safra                                                                     
2022          13      25        75    113  66.371681  22.123894  11.504425
2023          22      47       142    211  67.298578  22.274882  10.426540
2024          15      19        99    133  74.436090  14.285714  11.278195


In [29]:
volumetria_regiao = df_brasil_grupo6.groupby(['região', 'target']).size().unstack(fill_value=0)
volumetria_regiao = volumetria_regiao.reindex(columns=['Promotor', 'Neutro', 'Detrator'], fill_value=0)
volumetria_regiao['Total'] = volumetria_regiao.sum(axis=1)
volumetria_regiao['%Promotor'] = (volumetria_regiao['Promotor'] / volumetria_regiao['Total']) * 100
volumetria_regiao['%Neutro'] = (volumetria_regiao['Neutro'] / volumetria_regiao['Total']) * 100
volumetria_regiao['%Detrator'] = (volumetria_regiao['Detrator'] / volumetria_regiao['Total']) * 100
print("\nVolumetria por Região:")
print(volumetria_regiao)



Volumetria por Região:
target        Promotor  Neutro  Detrator  Total  %Promotor    %Neutro  \
região                                                                  
centro-oeste       168      37        26    231  72.727273  16.017316   
desconhecido        19       7         7     33  57.575758  21.212121   
nordeste            23      14         1     38  60.526316  36.842105   
norte               26      15         0     41  63.414634  36.585366   
sudeste             42       9         6     57  73.684211  15.789474   
sul                 38       9        10     57  66.666667  15.789474   

target        %Detrator  
região                   
centro-oeste  11.255411  
desconhecido  21.212121  
nordeste       2.631579  
norte          0.000000  
sudeste       10.526316  
sul           17.543860  


Estados Desconhecidos: ['**']


Regiões diferentes no modelo: ['sudeste' 'desconhecido' 'nordeste']
